In [163]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/thesis2/merged.csv


In [164]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
np.random.seed(42)
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import nltk, string, re, spacy,unicodedata, random
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
CPU times: user 67.7 ms, sys: 19.2 ms, total: 86.9 ms
Wall time: 109 ms


In [165]:
import pandas as pd
data = pd.read_csv("/kaggle/input/thesis2/merged.csv")
print(f'Total number of Documents: {len(data)}')

Total number of Documents: 7300


In [166]:
import pandas as pd

# Assuming your dataset is stored in a DataFrame called 'df'
# Replace 'clean text' with the actual column name in your dataset
duplicates = data[data.duplicated(subset='text', keep=False)]

# 'duplicates' now contains all rows where the 'clean text' column has duplicate values
# You can inspect these entries to understand why they are duplicated

# To print the count of duplicated rows:
print(f"Number of duplicated rows: {len(duplicates)}")

Number of duplicated rows: 0


In [167]:
data = data[['text','label']]

In [168]:
data = data.rename(columns={0:'text',1:'label'})

In [169]:
total_categories = data['label'].nunique()
print("Total number of categories:", total_categories)

Total number of categories: 6


In [170]:
# Assuming 'data' is your DataFrame with columns renamed
category_names = data['label'].unique()
print("Category names:", category_names)

Category names: ['Software Engineering' 'DBMS' 'Data Communication' 'Cyber Security '
 'Operating System' 'Cyber Security']


In [172]:
# Replace 'Cyber Security' and 'Cyber Security ' with 'Cyber Security'
data['label'] = data['label'].replace(['Cyber Security ', 'Cyber Security  '], 'Cyber Security')

# Replace 'Operating System' and 'operating System' with 'Operating System'
data['label'] = data['label'].replace(['Operating System', 'operating System'], 'Operating System')

In [173]:
# Assuming 'data' is your DataFrame with the 'label' column containing category names
unique_classes = data['label'].unique()
print("Unique classes:", unique_classes)

Unique classes: ['Software Engineering' 'DBMS' 'Data Communication' 'Cyber Security'
 'Operating System']


In [174]:
def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations

                           "]+", flags=re.UNICODE)
    #english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    #latin_pattern=re.compile('[A-Za-z\u00C0-\u00D6\u00D8-\u00f6\u00f8-\u00ff\s]*',)

    text=emoji_pattern.sub(r'', text)
    #text=english_pattern.sub(r'', text)
    text = re.sub(r'₀|₁|₂|₃|₄|₅|₆|₇|₈|₉', '', text)
    #text = re.sub(r'⁰|¹|²|³|⁴|⁵|⁶|⁷|⁸|⁹|⁻|⁺|ⁿ|ᵗ', '', text)

    return text

def remove_punctuations(my_str):
    # define punctuation
    punctuations = '''````£|¢|Ñ+-*/=৳০১২৩৪৫৬৭৮৯012–34567•89।!()-−[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰θ→°π∈∝α≤≥Δβε≠�￰৷￰'''

    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct



def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out=remove_punctuations(replace_strings(text))
    return out


In [175]:
data['cleaned'] = data['text'].apply(lambda x: preprocessing(str(x)))

In [176]:
sample_data = range(4000, 5000)
for i in sample_data:
    print(data['cleaned'][i])

গ্রে হ্যাট হ্যাকাররা  আবার তা নিজের স্বার্থের জন্যও ব্যবহার করতে পারে
ব্ল্যাক হ্যাট হ্যাকার Black Hat Hacker এই ধরনের হ্যাকাররা সবচেয়ে ভয়ংকর প্রকৃতির হ্যাকার 
ব্ল্যাক হ্যাট হ্যাকার  কোনো একটি সিকিউরিটি সিস্টেমের ত্রুটিগুলো বের করলে দ্রুত ঐ ত্রুটিকে নিজের স্বার্থে কাজে লাগায় 
ব্ল্যাক হ্যাট হ্যাকার  এরা সিস্টেম নষ্ট করে বিভিন্ন ভাইরাস ছড়িয়ে দেয় 
ব্ল্যাক হ্যাট হ্যাকার  তারা ভবিষ্যতে নিজে আবার যেন ঢুকতে পারে সে পথ রাখে 
ব্ল্যাক হ্যাট হ্যাকার  সর্বোপরি ঐ সিস্টেমের অধীনে যেসকল সাবসিস্টেম রয়েছে সেগুলোতেও ঢুকতে চেষ্টা করে
ব্লাক হ্যাট হ্যাকারদের কারণেই সবচেয়ে বেশি সাইবার ক্রাইম হয় এবং ইন্টারনেটে ধবংসাত্মক ভাইরাস এরাই তৈরি করে থাকে
ক্রাক্যাররাও  একধরনের হ্যাকার তবে এরা বিভিন্ন সিস্টেমে ঢুকে সেটির অনেক কিছু পরিবর্তন করে এবং কম্পিউটার ভাইরাস দিয়ে সকল তথ্য মুছে দেয় 
 অনেক সময় বিভিন্ন ওয়েবসাইট হ্যাক করে তার বিনিময়ে টাকা দাবি করা হয় এ ধরনের হ্যাকারকে ক্র্যাকার বলা হয়
পৃথিবীর বিভিন্ন দেশে যেসব সাইবার আইন করা হয়েছে তা মূলত এসব ক্র্যাকারদের জন্যই করা হয়েছে 
ক্র্যাকাররা বিভিন্নভাবে কাজ করে থাকে
তারা অনেক

In [177]:
stop =["অতএব","অথচ","অথবা","অনুযায়ী","অনেক","অনেকে","অনেকেই","অন্তত","অন্য","অবধি","অবশ্য","অর্থাত","আই","আগামী","আগে","আগেই","আছে","আজ","আদ্যভাগে","আপনার","আপনি","আবার","আমরা","আমাকে","আমাদের","আমার","আমি","আর","আরও","ই","ইত্যাদি","ইহা","উচিত","উত্তর","উনি","উপর","উপরে","এ","এঁদের","এঁরা","এই","একই","একটি","একবার","একে","এক্","এখন","এখনও","এখানে","এখানেই","এটা","এটাই","এটি","এত","এতটাই","এতে","এদের","এব","এবং","এবার","এমন","এমনকী","এমনি","এর","এরা","এল","এস","এসে","ঐ","ও","ওঁদের","ওঁর","ওঁরা","ওই","ওকে","ওখানে","ওদের","ওর","ওরা","কখনও","কত","কবে","কমনে","কয়েক","কয়েকটি","করছে","করছেন","করতে","করবে","করবেন","করলে","করলেন","করা","করাই","করায়","করার","করি","করিতে","করিয়া","করিয়ে","করে","করেই","করেছিলেন","করেছে","করেছেন","করেন","কাউকে","কাছ","কাছে","কাজ","কাজে","কারও","কারণ","কি","কিংবা","কিছু","কিছুই","কিন্তু","কী","কে","কেউ","কেউই","কেখা","কেন","কোটি","কোন","কোনও","কোনো","ক্ষেত্রে","কয়েক","খুব","গিয়ে","গিয়েছে","গিয়ে","গুলি","গেছে","গেল","গেলে","গোটা","চলে","চান","চায়","চার","চালু","চেয়ে","চেষ্টা","ছাড়া","ছাড়াও","ছিল","ছিলেন","জন","জনকে","জনের","জন্য","জন্যওজে","জানতে","জানা","জানানো","জানায়","জানিয়ে","জানিয়েছে","জে","জ্নজন","টি","ঠিক","তখন","তত","তথা","তবু","তবে","তা","তাঁকে","তাঁদের","তাঁর","তাঁরা","তাঁাহারা","তাই","তাও","তাকে","তাতে","তাদের","তার","তারপর","তারা","তারৈ","তাহলে","তাহা","তাহাতে","তাহার","তিনঐ","তিনি","তিনিও","তুমি","তুলে","তেমন","তো","তোমার","থাকবে","থাকবেন","থাকা","থাকায়","থাকে","থাকেন","থেকে","থেকেই","থেকেও","দিকে","দিতে","দিন","দিয়ে","দিয়েছে","দিয়েছেন","দিলেন","দু","দুই","দুটি","দুটো","দেওয়া","দেওয়ার","দেওয়া","দেখতে","দেখা","দেখে","দেন","দেয়","দ্বারা","ধরা","ধরে","ধামার","নতুন","নয়","না","নাই","নাকি","নাগাদ","নানা","নিজে","নিজেই","নিজেদের","নিজের","নিতে","নিয়ে","নিয়ে","নেই","নেওয়া","নেওয়ার","নেওয়া","নয়","পক্ষে","পর","পরে","পরেই","পরেও","পর্যন্ত","পাওয়া","পাচ","পারি","পারে","পারেন","পি","পেয়ে","পেয়্র্","প্রতি","প্রথম","প্রভৃতি","প্রযন্ত","প্রাথমিক","প্রায়","প্রায়","ফলে","ফিরে","ফের","বক্তব্য","বদলে","বন","বরং","বলতে","বলল","বললেন","বলা","বলে","বলেছেন","বলেন","বসে","বহু","বা","বাদে","বার","বি","বিনা","বিভিন্ন","বিশেষ","বিষয়টি","বেশ","বেশি","ব্যবহার","ব্যাপারে","ভাবে","ভাবেই","মতো","মতোই","মধ্যভাগে","মধ্যে","মধ্যেই","মধ্যেও","মনে","মাত্র","মাধ্যমে","মোট","মোটেই","যখন","যত","যতটা","যথেষ্ট","যদি","যদিও","যা","যাঁর","যাঁরা","যাওয়া","যাওয়ার","যাওয়া","যাকে","যাচ্ছে","যাতে","যাদের","যান","যাবে","যায়","যার","যারা","যিনি","যে","যেখানে","যেতে","যেন","যেমন","র","রকম","রয়েছে","রাখা","রেখে","লক্ষ","শুধু","শুরু","সঙ্গে","সঙ্গেও","সব","সবার","সমস্ত","সম্প্রতি","সহ","সহিত","সাথে","সাধারণ","সামনে","সি","সুতরাং","সে","সেই","সেখান","সেখানে","সেটা","সেটাই","সেটাও","সেটি","স্পষ্ট","স্বয়ং","হইতে","হইবে","হইয়া","হওয়া","হওয়ায়","হওয়ার","হচ্ছে","হত","হতে","হতেই","হন","হবে","হবেন","হয়","হয়তো","হয়নি","হয়ে","হয়েই","হয়েছিল","হয়েছে","হয়েছেন","হল","হলে","হলেই","হলেও","হলো","হাজার","হিসাবে","হৈলে","হোক","হয়"]
display(stop)

['অতএব',
 'অথচ',
 'অথবা',
 'অনুযায়ী',
 'অনেক',
 'অনেকে',
 'অনেকেই',
 'অন্তত',
 'অন্য',
 'অবধি',
 'অবশ্য',
 'অর্থাত',
 'আই',
 'আগামী',
 'আগে',
 'আগেই',
 'আছে',
 'আজ',
 'আদ্যভাগে',
 'আপনার',
 'আপনি',
 'আবার',
 'আমরা',
 'আমাকে',
 'আমাদের',
 'আমার',
 'আমি',
 'আর',
 'আরও',
 'ই',
 'ইত্যাদি',
 'ইহা',
 'উচিত',
 'উত্তর',
 'উনি',
 'উপর',
 'উপরে',
 'এ',
 'এঁদের',
 'এঁরা',
 'এই',
 'একই',
 'একটি',
 'একবার',
 'একে',
 'এক্',
 'এখন',
 'এখনও',
 'এখানে',
 'এখানেই',
 'এটা',
 'এটাই',
 'এটি',
 'এত',
 'এতটাই',
 'এতে',
 'এদের',
 'এব',
 'এবং',
 'এবার',
 'এমন',
 'এমনকী',
 'এমনি',
 'এর',
 'এরা',
 'এল',
 'এস',
 'এসে',
 'ঐ',
 'ও',
 'ওঁদের',
 'ওঁর',
 'ওঁরা',
 'ওই',
 'ওকে',
 'ওখানে',
 'ওদের',
 'ওর',
 'ওরা',
 'কখনও',
 'কত',
 'কবে',
 'কমনে',
 'কয়েক',
 'কয়েকটি',
 'করছে',
 'করছেন',
 'করতে',
 'করবে',
 'করবেন',
 'করলে',
 'করলেন',
 'করা',
 'করাই',
 'করায়',
 'করার',
 'করি',
 'করিতে',
 'করিয়া',
 'করিয়ে',
 'করে',
 'করেই',
 'করেছিলেন',
 'করেছে',
 'করেছেন',
 'করেন',
 'কাউকে',
 'কাছ',
 'কাছে',
 'কাজ',
 'কাজে',
 'কারও',
 '

In [179]:
def stopwordRemoval(text):
    x=str(text)
    l=x.split()

    stm=[elem for elem in l if elem not in stop]

    out=' '.join(stm)

    return str(out)

In [180]:
# Apply the stopwordRemoval function to the 'clean text' column
data['cleaned'] = data['cleaned'].apply(lambda x: stopwordRemoval(str(x)))

# Iterate through the first 1000 rows and print the 'clean text' column values
sample_data = range(0, 1000)
for i in sample_data:
    print(data['cleaned'][i])

সফটওয়্যার ইঞ্জিনিয়ারিং দুইটি শব্দ সফটওয়্যার ইঞ্জিনিয়ারিংএর সমন্বয়ে গঠিত
সফটওয়্যার প্রোগ্রামের সমষ্টি
প্রোগ্রাম নির্ধারিত সমস্যা সমাধানের উদ্দেশ্যে তৈরি সফটওয়্যার প্রোডাক্ট
অপরদিকে ইঞ্জিনিয়ারিং সকল নীতিমালা মেথড সমূহ প্রোডাক্টটি Software ফিনিশিং প্রোডাক্ট সেগুলোর সুশৃঙ্খল প্রয়োগ করাকে বুঝায়
সফটওয়্যার ইঞ্জিনিয়ারিং বুঝায়
সফটওয়্যার ইঞ্জিনিয়ারিং ইঞ্জিনিয়ারিং শাখা সফটওয়্যার প্রডাক্ট ডেভেলপমেন্টের সুনির্ধারিত বৈজ্ঞানিক নীতিমালা Scientific principle মেথড ব্যবহৃত প্রক্রিয়া Procedures সম্পর্কিত
সফটওয়্যার ইঞ্জিনিয়ারিংএর outcome দক্ষ Efficient নির্ভরযোগ্য Reliable সফটওয়্যার প্রোডাক্ট
অর্থাৎ ডেভেলপমেন্ট অপারেশন সফটওয়্যার সংরক্ষণের সুষম শৃঙ্খলাবদ্ধ quantifiable পদ্ধতির সফল প্রয়োগই সফটওয়্যার ইঞ্জিনিয়ারিং
জার্মান কম্পিউটার বিজ্ঞানী Fritz Bauer মতে প্রকৃত মেশিনসমূহে কার্যকরিভাবে সাউন্ড ইঞ্জিনিয়ারিং Core Engineeringএর নীতিসমূহ সফটওয়্যার সমূহে সুষ্ঠ ব্যবহারই সফটওয়্যার ইঞ্জিনিয়ারিং
পরিশেষে পূর্ণ সফটওয়্যার ডিজাইন ডেভেলপমেন্ট রক্ষণাবেক্ষণের ইঞ্জিনিয়ারিং শাখা সফটওয়্যার ইঞ্জিনিয়ার

In [181]:
# Split the data into training, validation, and testing sets
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

# Print the shapes of the resulting datasets
print("Training data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)
print("Testing data shape:", test_data.shape)

Training data shape: (5110, 3)
Validation data shape: (1095, 3)
Testing data shape: (1095, 3)


In [182]:
train_data['enc_label'] = train_data['label'].replace({'Software Engineering':0, 'DBMS':1, 'Data Communication':2 ,'Cyber Security':3,
 'Operating System':4})
val_data['enc_label'] = val_data['label'].replace({'Software Engineering':0, 'DBMS':1, 'Data Communication':2 ,'Cyber Security':3,
 'Operating System':4})

In [183]:
train_data['cleaned'].tolist()

['ধরনের পরিস্থিতি মোকাবেলা দরকার ইন্টেলেকচুয়াল প্রপার্টি আইনের প্রয়োগ',
 'কম্পিউটার ভাইরাস মূলত প্রোগ্রামারদের ডেভেলপ ক্ষতিকর প্রোগ্রাম',
 'তাছাড়া ভাটা দ্রুত পড়া Read যায়',
 'প্রযুক্তির অনুমতি প্রক্রিয়াকরণ ট্রান্সপন্ডার ত্রুটি কর্মক্ষমতা ডিজিটাল সিগন্যালিংয়ের উন্নতি উপলব্ধি ব্যবহারে আসা',
 'ফাইল মুছে ফেলার নামযুক্ত ফাইলের ডিরেক্টরি অনুসন্ধান',
 'আরেক দল ক্র্যাকার সফটওয়্যার টুলের সিস্টেমে অবৈধভাবে প্রবেশ',
 'পাসিং রেফারেন্স বাদ কথোপকথনের অংশ সফ্টওয়্যার কমই উল্লেখ',
 'মডুলেটেড ক্যারিয়ার ওয়েভের প্রেক্ষিতেও মডুলেশন ইনডেক্সকে সংজ্ঞায়িত যায়',
 'অবজেক্ট অরিয়েন্টেড ডাটা মডেল মূলত অবজেক্ট অরিয়েন্টেড প্রোগ্রামিং ল্যাংগুয়েজের ধারণার প্রতিষ্ঠিত',
 'সিস্টেমটি ব্যবহারযোগ্য দক্ষতার ডেটা পুনরুদ্ধার',
 'কমান্ড পেলে প্রোগ্রাম ডিস্ক মেমোরিতে পূর্বের প্রোগ্রাম ওভাররাইট',
 'সিগন্যালের সেকেন্ডে ডাটা ট্রান্সফারের হারকে Data Rate',
 'কোড লেখার সময় প্রোগ্রামারগণ কোড উদ্দেশ্য কারা লিখছে কোথায় প্রয়োজন',
 'উপাদানস্তরের নকশা সফ্টওয়্যার আর্কিটেকচারের কাঠামোগত উপাদানগুলিকে সফ্টওয়্যার উপাদানগুলির পদ্ধতিগত বিবরণে

In [184]:
train_data['enc_label'].value_counts()

enc_label
3    1122
2    1084
0    1004
1     973
4     927
Name: count, dtype: int64

In [186]:
val_data['enc_label'].value_counts()

enc_label
2    228
0    225
3    219
1    212
4    211
Name: count, dtype: int64

In [187]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [188]:
train_data = train_data.rename(columns={'label_Software Engineering':'Software Engineering','label_DBMS':'DBMS',
                                        'label_Data Communication':'Data Communication','label_Cyber Security':'Cyber Security','label_Operating System':'Operating System'})

train_data

,text,label,cleaned,enc_label
4041,এই ধরনের পরিস্থিতি মোকাবেলা করতে দরকার হয় ইন্ট...,Cyber Security,ধরনের পরিস্থিতি মোকাবেলা দরকার ইন্টেলেকচুয়াল প...,3
4258,কম্পিউটার ভাইরাস মূলত প্রোগ্রামারদের ডেভেলপ কর...,Cyber Security,কম্পিউটার ভাইরাস মূলত প্রোগ্রামারদের ডেভেলপ ক্...,3
1748,তাছাড়া এটি হতে ভাটা দ্রুত পড়া (Read) যায়।,DBMS,তাছাড়া ভাটা দ্রুত পড়া Read যায়,1
6581,"প্রযুক্তির অনুমতি হিসাবে, প্রক্রিয়াকরণ ট্রান্...",Data Communication,প্রযুক্তির অনুমতি প্রক্রিয়াকরণ ট্রান্সপন্ডার ...,2
6911,"একটি ফাইল মুছে ফেলার জন্য, আমরা নামযুক্ত ফাইলে...",Operating System,ফাইল মুছে ফেলার নামযুক্ত ফাইলের ডিরেক্টরি অনুস...,4
...,...,...,...,...
5191,স্বয়ংক্রিয়ভাবে এক জব থেকে অন্য জবে যাওয়ার উপর ...,Operating System,স্বয়ংক্রিয়ভাবে এক জব জবে যাওয়ার ভিত্তি অপারেটি...,4
5226,সার্ভার প্রথমে ক্লায়েন্টের নিকট হতে নির্দিষ্ট ...,Operating System,সার্ভার প্রথমে ক্লায়েন্টের নিকট নির্দিষ্ট Part...,4
5390,যদি কোনো System-এ Deadlock prevention বা Deadl...,Operating System,Systemএ Deadlock prevention Deadlock avoidance...,4
860,কম্পিউটার প্রোগ্রামিং ভাষা প্রয়োগের মাধ্যমে সফ...,Software Engineering,কম্পিউটার প্রোগ্রামিং ভাষা প্রয়োগের সফটওয়্যার ...,0


In [189]:
def char_length(row):
    return len(row.split())

In [190]:
lables = ['Software Engineering' 'DBMS' 'Data Communication' 'Cyber Security'
 'Operating System']

# Count number of unique words in each set
def dataset_statistics(dataset):
  for l in lables:
    word_list = [word.strip() for t in list(dataset[dataset[l]==1.0].cleaned) for word in t.strip().split()]
    print(f"\nClass Name:--> {l} ")
    print("Number of Words:{}".format(len(word_list)))
    print("Number of Unique Words:{}".format(len(np.unique(word_list))))
    print("Maximum post length: {}".format(max(dataset[dataset[l]==1.0].cleaned.apply(char_length))))
    print("Average Words in texts: {}".format(np.mean(dataset[dataset[l]==1.0].cleaned.apply(char_length))))

In [191]:
# Class Distribution
class_distribution = train_data['label'].value_counts()

# Text Length Distribution
train_data['text_length'] = train_data['cleaned'].apply(lambda x: len(x.split()))
text_length_stats = train_data['text_length'].describe()

print("Class Distribution:")
print(class_distribution)
print("\nText Length Distribution:")
print(text_length_stats)

Class Distribution:
label
Cyber Security          1122
Data Communication      1084
Software Engineering    1004
DBMS                     973
Operating System         927
Name: count, dtype: int64

Text Length Distribution:
count    5110.000000
mean        9.935225
std         4.734930
min         1.000000
25%         7.000000
50%         9.000000
75%        12.000000
max        59.000000
Name: text_length, dtype: float64


In [192]:
categories = ['Software Engineering', 'DBMS' ,'Data Communication', 'Cyber Security',
 'Operating System']

In [193]:
def print_metrices(pred,true):
    print(confusion_matrix(true,pred))
    print(classification_report(true,pred,))
    print("Accuracy : ",accuracy_score(pred,true))
    print("Precison : ",precision_score(pred,true, average = 'weighted'))
    print("Recall : ",recall_score(pred,true,  average = 'weighted'))
    print("F1 : ",f1_score(pred,true,  average = 'weighted'))

In [194]:
labels = ['Software Engineering', 'DBMS' ,'Data Communication', 'Cyber Security',
 'Operating System']
def actual_label(row):
    return labels[row]

In [197]:
X_train = train_data['cleaned'].tolist()
y_train = train_data['label'].tolist()

X_valid = val_data['cleaned'].tolist()
y_valid = val_data['label'].tolist()

X_test = test_data['cleaned'].tolist()
y_test = test_data['label'].tolist()

'Logistic Regression','Decision Tree','Random Forest','Naive Bayes', 'SVM', 'KNN', 'AdaBoost']

In [198]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the vectorizer
vectorizer = TfidfVectorizer()

# Convert text data to numerical data (fit_transform on training data and transform on test data)
X_train_transformed = vectorizer.fit_transform(X_train)
X_valid_transformed = vectorizer.transform(X_valid)  # Transform validation data
X_test_transformed = vectorizer.transform(X_test)

# Initialize logistic regression model
model = LogisticRegression(random_state=42)

# Train the model
model.fit(X_train_transformed, y_train)

# Predictions on validation set
y_valid_pred = model.predict(X_valid_transformed)

# Evaluate on validation set
print("Validation Set Classification Report:")
print(classification_report(y_valid, y_valid_pred))

# Predictions on test set
y_test_pred = model.predict(X_test_transformed)

# Evaluate on test set
print("Test Set Classification Report:")
print(classification_report(y_test, y_test_pred))


Validation Set Classification Report:
                      precision    recall  f1-score   support

      Cyber Security       0.74      0.74      0.74       219
                DBMS       0.71      0.72      0.72       212
  Data Communication       0.74      0.80      0.77       228
    Operating System       0.78      0.75      0.76       211
Software Engineering       0.82      0.76      0.79       225

            accuracy                           0.76      1095
           macro avg       0.76      0.76      0.76      1095
        weighted avg       0.76      0.76      0.76      1095

Test Set Classification Report:
                      precision    recall  f1-score   support

      Cyber Security       0.78      0.78      0.78       243
                DBMS       0.70      0.65      0.68       216
  Data Communication       0.75      0.81      0.78       222
    Operating System       0.80      0.79      0.79       215
Software Engineering       0.77      0.79      0.78       

In [200]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the vectorizer
vectorizer = TfidfVectorizer()

# Convert text data to numerical data (fit_transform on training data and transform on test data)
X_train_transformed = vectorizer.fit_transform(X_train)
X_valid_transformed = vectorizer.transform(X_valid)  # Transform validation data
X_test_transformed = vectorizer.transform(X_test)

classifiers = {
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Naive Bayes': MultinomialNB(),
    'SVM': SVC(random_state=42),
    'KNN': KNeighborsClassifier(),
    'AdaBoost': AdaBoostClassifier(random_state=42)
}

for name, classifier in classifiers.items():
    print(f"\n{name}:")
    # Train the model
    classifier.fit(X_train_transformed, y_train)

    # Predictions on validation set
    y_valid_pred = classifier.predict(X_valid_transformed)

    # Evaluate on validation set
    print("Validation Set Classification Report:")
    print(classification_report(y_valid, y_valid_pred))

    # Predictions on test set
    y_test_pred = classifier.predict(X_test_transformed)

    # Evaluate on test set
    print("Test Set Classification Report:")
    print(classification_report(y_test, y_test_pred))



Decision Tree:
Validation Set Classification Report:
                      precision    recall  f1-score   support

      Cyber Security       0.59      0.66      0.62       219
                DBMS       0.57      0.55      0.56       212
  Data Communication       0.68      0.73      0.70       228
    Operating System       0.71      0.63      0.66       211
Software Engineering       0.71      0.68      0.70       225

            accuracy                           0.65      1095
           macro avg       0.65      0.65      0.65      1095
        weighted avg       0.65      0.65      0.65      1095

Test Set Classification Report:
                      precision    recall  f1-score   support

      Cyber Security       0.62      0.65      0.64       243
                DBMS       0.57      0.52      0.55       216
  Data Communication       0.63      0.64      0.63       222
    Operating System       0.68      0.65      0.66       215
Software Engineering       0.64      0.68 

In [201]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the vectorizer
vectorizer = CountVectorizer()

# Convert text data to numerical data (fit_transform on training data and transform on test data)
X_train_transformed = vectorizer.fit_transform(X_train)
X_valid_transformed = vectorizer.transform(X_valid)  # Transform validation data
X_test_transformed = vectorizer.transform(X_test)

classifiers = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Naive Bayes': MultinomialNB(),
    'SVM': SVC(random_state=42),
    'KNN': KNeighborsClassifier(),
    'AdaBoost': AdaBoostClassifier(random_state=42)
}

for name, classifier in classifiers.items():
    print(f"\n{name}:")
    # Train the model
    classifier.fit(X_train_transformed, y_train)

    # Predictions on validation set
    y_valid_pred = classifier.predict(X_valid_transformed)

    # Evaluate on validation set
    print("Validation Set Classification Report:")
    print(classification_report(y_valid, y_valid_pred))

    # Predictions on test set
    y_test_pred = classifier.predict(X_test_transformed)

    # Evaluate on test set
    print("Test Set Classification Report:")
    print(classification_report(y_test, y_test_pred))



Logistic Regression:
Validation Set Classification Report:
                      precision    recall  f1-score   support

      Cyber Security       0.76      0.70      0.73       219
                DBMS       0.65      0.73      0.69       212
  Data Communication       0.70      0.78      0.74       228
    Operating System       0.79      0.72      0.75       211
Software Engineering       0.82      0.76      0.79       225

            accuracy                           0.74      1095
           macro avg       0.74      0.74      0.74      1095
        weighted avg       0.74      0.74      0.74      1095

Test Set Classification Report:
                      precision    recall  f1-score   support

      Cyber Security       0.77      0.73      0.75       243
                DBMS       0.66      0.67      0.67       216
  Data Communication       0.72      0.78      0.75       222
    Operating System       0.79      0.76      0.78       215
Software Engineering       0.77     